In [3]:
import pandas as pd
import os
import re # สำหรับจัดการ Regular Expressions

# --- 1. กำหนด Path ของไฟล์ ---

# Path ไฟล์ Input (ข้อมูลดิบ)
canal_file_path = '../data/RAW/canel_list.xls'
district_file_path = '../data/PROCESSED/overall_data.xlsx'

# Path ไฟล์ Output (ผลลัพธ์ที่จะบันทึก)
output_dir = '../data/PROCESSED/'
output_filename = 'canals_with_dcode.csv'
output_path = os.path.join(output_dir, output_filename)

# ชื่อชีตที่เราจะดึงข้อมูล
canal_sheet_name = 'ข้อมูลคลองเรียงตามตัวอักษร'
district_sheet_name = 'districts'

print("--- 🚀 เริ่มกระบวนการ V.4 (โหลด, ทำความสะอาด, รวม, บันทึก) ---")


# --- 2. ฟังก์ชันทำความสะอาด V4 (หัวใจของงาน) ---
def normalize_district_name_v4(name):
    """
    ฟังก์ชัน V4: รวมทุกขั้นตอนไว้ในที่เดียว
    1. เปลี่ยนเป็น string
    2. ลบ "เขต"
    3. ลบ Non-breaking space (u'\xa0')
    4. แก้การสะกดผิด 'ราษฏร์บูรณะ' (ฏ) -> 'ราษฎร์บูรณะ' (ฎ)
    5. ลบช่องว่าง "ทุกชนิด" ( \s+ ) ทิ้งทั้งหมด
    """
    if pd.isna(name):
        return None
    
    clean_name = str(name)
    clean_name = clean_name.replace('เขต', '')
    clean_name = clean_name.replace(u'\xa0', '')
    clean_name = clean_name.replace('ราษฏร์บูรณะ', 'ราษฎร์บูรณะ') # แก้คำผิด
    clean_name = re.sub(r'\s+', '', clean_name) # ลบ space ทั้งหมด
    
    return clean_name


try:
    # --- 3. โหลดและเตรียมข้อมูลเขต (Districts) ---
    print(f"📖 กำลังโหลดไฟล์เขต: {district_file_path} (ชีต: {district_sheet_name})")
    df_districts = pd.read_excel(district_file_path, sheet_name=district_sheet_name)
    df_district_map = df_districts[['dcode', 'dname']].copy()
    
    # สร้าง 'join_key' โดยใช้ฟังก์ชัน V4
    df_district_map['join_key'] = df_district_map['dname'].apply(normalize_district_name_v4)
    print("   🗺️ ตารางเขต (join_key V4) สร้างแล้ว")


    # --- 4. โหลดและเตรียมข้อมูลคลอง (Canals) ---
    print(f"🌊 กำลังโหลดไฟล์คลอง: {canal_file_path} (ชีต: {canal_sheet_name})")
    # header=2 เพราะข้อมูลจริงเริ่มที่แถวที่ 3
    df_canals_raw = pd.read_excel(canal_file_path, sheet_name=canal_sheet_name, header=2) 
    df_canals = df_canals_raw[['ชื่อคลอง', 'พื้นที่เขต']].copy()
    
    # ลบแถวที่ข้อมูลหลักเป็นค่าว่าง
    df_canals = df_canals.dropna(subset=['ชื่อคลอง', 'พื้นที่เขต'])
    
    # สร้าง 'join_key' โดยใช้ฟังก์ชัน V4 "เดียวกัน"
    df_canals['join_key'] = df_canals['พื้นที่เขต'].apply(normalize_district_name_v4)
    print("   🌊 ตารางคลอง (join_key V4) สร้างแล้ว")


    # --- 5. รวม (Merge) ข้อมูล ---
    print("\n" + "="*50)
    print("🤝 กำลังรวมข้อมูลคลองและเขต (ด้วย join_key V4)...")
    
    df_final = pd.merge(
        df_canals,
        df_district_map,
        on='join_key',  # ใช้ Key ที่สร้างด้วยกระบวนการเดียวกัน 100%
        how='left'      # 'left' = เก็บคลองไว้ทั้งหมด
    )
    
    # เลือกเฉพาะคอลัมน์ผลลัพธ์ที่ต้องการ (ไม่เอา join_key ไปด้วย)
    df_result = df_final[['ชื่อคลอง', 'พื้นที่เขต', 'dcode', 'dname']]
    
    # ตรวจสอบผลลัพธ์ (สำคัญมาก)
    unmatched_count = df_result['dcode'].isna().sum()
    print(f"✅ รวมข้อมูลเสร็จสิ้น! จำนวนคลองที่หา DCODE ไม่พบ: {unmatched_count}")


    # --- 6. บันทึกไฟล์ (Save) ---
    if unmatched_count == 0:
        print("\n" + "="*50)
        print(f"💾 กำลังบันทึกไฟล์ผลลัพธ์ที่สมบูรณ์...")
        
        # สร้างโฟลเดอร์ PROCESSED (ถ้ายังไม่มี)
        os.makedirs(output_dir, exist_ok=True)
        
        # บันทึกเป็น CSV
        # index=False คือไม่เอาเลข index (0,1,2..) ของ DataFrame ไปใส่ในไฟล์
        # encoding='utf-8-sig' เพื่อให้ Excel เปิดภาษาไทยได้ถูกต้อง
        df_result.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print(f"🎉 บันทึกไฟล์สำเร็จ!")
        print(f"   ไฟล์: {output_filename}")
        print(f"   ที่อยู่: {output_path}")
    
    else:
        print("❌ [คำเตือน] ยังมีข้อมูลบางส่วนที่หา DCODE ไม่พบ")
        print("   กรุณาตรวจสอบข้อมูลก่อน และจะยังไม่บันทึกไฟล์อัตโนมัติ")
        unmatched_keys = df_result[df_result['dcode'].isna()]['join_key'].unique()
        print(f"   Join Keys ที่ยังคงหาไม่เจอ: {unmatched_keys[:20]}")


except FileNotFoundError as e:
    print(f"❌ เกิดข้อผิดพลาด: ไม่พบไฟล์")
    print(f"กรุณาตรวจสอบว่าไฟล์นี้มีอยู่จริง: {e.filename}")
except ValueError as e:
    print(f"❌ เกิดข้อผิดพลาด: ไม่พบชื่อชีต (Sheet)")
    if 'Worksheet' in str(e):
        print(f"ไม่พบชีต '{canal_sheet_name}' หรือ '{district_sheet_name}'")
    else:
        print(e)
except KeyError as e:
    print(f"❌ เกิดข้อผิดพลาด: ไม่พบคอลัมน์")
    print(f"ไม่พบคอลัมน์ชื่อ {e} กรุณาตรวจสอบ Header ในไฟล์ Excel")
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดไม่คาดคิด: {e}")

--- 🚀 เริ่มกระบวนการ V.4 (โหลด, ทำความสะอาด, รวม, บันทึก) ---
📖 กำลังโหลดไฟล์เขต: ../data/PROCESSED/overall_data.xlsx (ชีต: districts)
   🗺️ ตารางเขต (join_key V4) สร้างแล้ว
🌊 กำลังโหลดไฟล์คลอง: ../data/RAW/canel_list.xls (ชีต: ข้อมูลคลองเรียงตามตัวอักษร)
   🌊 ตารางคลอง (join_key V4) สร้างแล้ว

🤝 กำลังรวมข้อมูลคลองและเขต (ด้วย join_key V4)...
✅ รวมข้อมูลเสร็จสิ้น! จำนวนคลองที่หา DCODE ไม่พบ: 0

💾 กำลังบันทึกไฟล์ผลลัพธ์ที่สมบูรณ์...
🎉 บันทึกไฟล์สำเร็จ!
   ไฟล์: canals_with_dcode.csv
   ที่อยู่: ../data/PROCESSED/canals_with_dcode.csv


In [4]:
import pandas as pd
import os
import glob
import re  # สำหรับ Regular Expressions
import time # สำหรับจับเวลา

# --- 1. [V4.2] ฟังก์ชันทำความสะอาดชื่อเขต (อัปเกรด) ---
def normalize_district_name_v4_2(name):
    """
    ฟังก์ชัน V4.2:
    1. แก้ไขคำย่อ/สะกดผิด ที่พบบ่อย (ป้อมปราบฯ, ราษฏร์บูรณะ)
    2. ลบคำนำหน้าที่ไม่ต้องการ (เขต, อำเภอ)
    3. ลบอักขระที่มองไม่เห็น (\xa0)
    4. ลบช่องว่างทุกชนิด ( \s+ ) ทิ้งทั้งหมด
    """
    if pd.isna(name):
        return None
    
    clean_name = str(name)
    
    # --- 1. จัดการคำย่อ/สะกดผิด ก่อน ---
    clean_name = clean_name.replace('ป้อมปราบฯ', 'ป้อมปราบศัตรูพ่าย') # แก้คำย่อ
    clean_name = clean_name.replace('ราษฏร์บูรณะ', 'ราษฎร์บูรณะ')    # แก้ ฏ -> ฎ
    
    # --- 2. ลบคำนำหน้า ---
    clean_name = clean_name.replace('เขต', '')
    clean_name = clean_name.replace('อำเภอ', '') # สำหรับข้อมูลนอก กทม.
    
    # --- 3. ลบอักขระที่มองไม่เห็น ---
    clean_name = clean_name.replace(u'\xa0', '')
    
    # --- 4. ลบช่องว่างทั้งหมด ---
    clean_name = re.sub(r'\s+', '', clean_name)
    
    return clean_name

# --- 2. เตรียม "แผนที่เขต" (District Map) ---
print("--- 🗺️ ขั้นตอนที่ 1: กำลังสร้างตารางจับคู่ DCODE ---")
try:
    district_file_path = '../data/PROCESSED/overall_data.xlsx'
    district_sheet_name = 'districts'
    
    df_districts = pd.read_excel(district_file_path, sheet_name=district_sheet_name)
    df_district_map = df_districts[['dcode', 'dname']].copy()
    
    # ใช้ฟังก์ชัน V4.2 ใหม่
    df_district_map['join_key'] = df_district_map['dname'].apply(normalize_district_name_v4_2)
    df_district_map = df_district_map.drop_duplicates(subset=['join_key'])
    
    print(f"✅ สร้างตารางจับคู่ DCODE สำเร็จ (มี {len(df_district_map)} เขตที่ไม่ซ้ำกัน)")

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดในการโหลด dcode map: {e}")
    # (หยุดการทำงานถ้าโหลดแผนที่เขตไม่สำเร็จ)
    raise e


# --- 3. ค้นหาและประมวลผลไฟล์ฝน 12 เดือน ---
print("\n--- 🌦️ ขั้นตอนที่ 2: เริ่มประมวลผลไฟล์ข้อมูลฝน ---")

rain_data_dir = '../data/RAW/rain_2024/'

# 1. ค้นหาไฟล์ .xlsx ทั้งหมด (อาจจะเจอไฟล์ temp)
all_xlsx_files = glob.glob(os.path.join(rain_data_dir, '*.xlsx'))

# 2. [V4.1] กรองไฟล์ temporary ของ Excel (ที่ขึ้นต้นด้วย ~$) ออก
rain_files = sorted(
    [f for f in all_xlsx_files if not os.path.basename(f).startswith('~$')]
)

if not rain_files:
    print(f"❌ ไม่พบไฟล์ .xlsx ที่เป็นข้อมูลจริง ในโฟลเดอร์: {rain_data_dir}")
else:
    print(f"📂 พบไฟล์ข้อมูลฝน {len(rain_files)} ไฟล์ (กรองไฟล์ Temp แล้ว). กำลังเริ่มวนลูป...")


all_processed_data = [] # ลิสต์สำหรับเก็บ DataFrame ของทุกวัน
start_time = time.time()

try:
    # Loop 1: วนลูปไฟล์ (ที่กรองแล้ว)
    for file_path in rain_files:
        file_name = os.path.basename(file_path)
        print(f"\n   กำลังประมวลผลไฟล์: {file_name}...")
        
        dfs_by_day = pd.read_excel(file_path, sheet_name=None)
        
        # Loop 2: วนลูป ~30 ชีต (วัน)
        for sheet_name, df_day in dfs_by_day.items():
            
            if df_day.empty or 'เขต' not in df_day.columns:
                print(f"      - (ข้าม) ชีต '{sheet_name}' ว่างเปล่า หรือไม่มีคอลัมน์ 'เขต'")
                continue
            
            # --- Transform & Merge ---
            # ใช้ฟังก์ชัน V4.2 ใหม่
            df_day['join_key'] = df_day['เขต'].apply(normalize_district_name_v4_2)
            
            df_merged = pd.merge(
                df_day,
                df_district_map,
                on='join_key',
                how='left'
            )
            all_processed_data.append(df_merged)
            
        print(f"   ✅ {file_name} (รวม {len(dfs_by_day)} ชีต) ประมวลผลเสร็จสิ้น")


    # --- 4. รวมร่าง (Concat) ข้อมูลทั้งหมด ---
    if not all_processed_data:
        print("❌ ไม่พบข้อมูลใดๆ ให้รวมเลย!")
    else:
        print("\n" + "="*50)
        print("...กำลังรวมข้อมูลทั้งหมดจาก 365+ ชีต...")
        
        final_rain_df = pd.concat(all_processed_data, ignore_index=True)
        
        end_time = time.time()
        print(f"✅ รวมข้อมูลทั้งหมดเสร็จสิ้น! (ใช้เวลา {end_time - start_time:.2f} วินาที)")
        print(f"   ข้อมูลทั้งหมด: {final_rain_df.shape[0]} แถว, {final_rain_df.shape[1]} คอลัมน์")

        # --- 5. ตรวจสอบและบันทึก ---
        unmatched_rows = final_rain_df[final_rain_df['dcode'].isna()]
        unmatched_count = len(unmatched_rows)
        
        if unmatched_count > 0:
            print(f"\n   ⚠️ คำเตือน: มี {unmatched_count} แถวที่หา DCODE ไม่พบ")
            unmatched_districts = unmatched_rows['เขต'].unique()
            print(f"   ตัวอย่างชื่อเขตที่หาไม่พบ: {unmatched_districts[:10]}")
            
            # ตรวจสอบว่า 'ป้อมปราบฯ' ยังหลงเหลืออยู่หรือไม่
            if any('ป้อมปราบฯ' in s for s in unmatched_districts if isinstance(s, str)):
                 print("   [!] ยังพบ 'ป้อมปราบฯ' ที่ยังไม่ถูกแก้ไข!")
            else:
                 print("   [i] (ข้อมูล) 'ป้อมปราบฯ' ถูกแก้ไขเรียบร้อยแล้ว")
                 
            # ให้ข้อมูลเกี่ยวกับข้อมูลนอกพื้นที่
            if any('สมุทรปราการ' in s for s in unmatched_districts if isinstance(s, str)):
                print("   [i] (ข้อมูล) 'อำเภอเมืองสมุทรปราการ' ถูกตรวจพบ ซึ่งเป็นข้อมูลนอกพื้นที่ กทม. (นี่คือพฤติกรรมที่ถูกต้อง)")

        else:
            print("\n   ✅ DCODE ทั้งหมดถูกจับคู่เรียบร้อย!")
            
        # --- 6. จัดเรียงคอลัมน์และบันทึกไฟล์ ---
        if 'dcode' in final_rain_df.columns:
            key_cols = ['dcode', 'dname', 'เขต', 'รหัสสถานี', 'สถานี', 'วัน-เวลา']
            other_cols = [col for col in final_rain_df.columns if col not in key_cols and col != 'join_key']
            final_rain_df_sorted = final_rain_df[key_cols + other_cols]

        output_dir = '../data/PROCESSED/'
        output_filename = 'rain_2024_combined_with_dcode.csv'
        output_path = os.path.join(output_dir, output_filename)
        
        os.makedirs(output_dir, exist_ok=True)
        
        print(f"\n💾 กำลังบันทึกไฟล์ผลลัพธ์ไปที่: {output_path}")
        final_rain_df_sorted.to_csv(output_path, index=False, encoding='utf-8-sig')
        print("🎉 บันทึกไฟล์สำเร็จ!")

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดร้ายแรงระหว่างประมวลผล: {e}")

--- 🗺️ ขั้นตอนที่ 1: กำลังสร้างตารางจับคู่ DCODE ---
✅ สร้างตารางจับคู่ DCODE สำเร็จ (มี 50 เขตที่ไม่ซ้ำกัน)

--- 🌦️ ขั้นตอนที่ 2: เริ่มประมวลผลไฟล์ข้อมูลฝน ---
📂 พบไฟล์ข้อมูลฝน 12 ไฟล์ (กรองไฟล์ Temp แล้ว). กำลังเริ่มวนลูป...

   กำลังประมวลผลไฟล์: 01.xlsx...
   ✅ 01.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 02.xlsx...
   ✅ 02.xlsx (รวม 29 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 03.xlsx...
   ✅ 03.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 04.xlsx...
   ✅ 04.xlsx (รวม 30 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 05.xlsx...
   ✅ 05.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 06.xlsx...
   ✅ 06.xlsx (รวม 30 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 07.xlsx...
   ✅ 07.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 08.xlsx...
   ✅ 08.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 09.xlsx...
   ✅ 09.xlsx (รวม 30 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 10.xlsx...
   ✅ 10.xlsx (รวม 31 ชีต) ประมวลผลเสร

In [5]:
import pandas as pd
import os
import glob
import re  # สำหรับ Regular Expressions
import time # สำหรับจับเวลา

# --- 1. [V4.2] ฟังก์ชันทำความสะอาดชื่อเขต (คงเดิม) ---
def normalize_district_name_v4_2(name):
    """
    ฟังก์ชัน V4.2:
    1. แก้ไขคำย่อ/สะกดผิด (ป้อมปราบฯ, ราษฏร์บูรณะ)
    2. ลบคำนำหน้า (เขต, อำเภอ)
    3. ลบอักขระที่มองไม่เห็น (\xa0)
    4. ลบช่องว่างทุกชนิด ( \s+ ) ทิ้งทั้งหมด
    """
    if pd.isna(name):
        return None
    
    clean_name = str(name)
    clean_name = clean_name.replace('ป้อมปราบฯ', 'ป้อมปราบศัตรูพ่าย') # แก้คำย่อ
    clean_name = clean_name.replace('ราษฏร์บูรณะ', 'ราษฎร์บูรณะ')    # แก้ ฏ -> ฎ
    clean_name = clean_name.replace('เขต', '')
    clean_name = clean_name.replace('อำเภอ', '') # สำหรับข้อมูลนอก กทม.
    clean_name = clean_name.replace(u'\xa0', '')
    clean_name = re.sub(r'\s+', '', clean_name)
    
    return clean_name

# --- 2. เตรียม "แผนที่เขต" (District Map) ---
print("--- 🗺️ ขั้นตอนที่ 1: กำลังสร้างตารางจับคู่ DCODE ---")
try:
    district_file_path = '../data/PROCESSED/overall_data.xlsx'
    district_sheet_name = 'districts'
    
    df_districts = pd.read_excel(district_file_path, sheet_name=district_sheet_name)
    df_district_map = df_districts[['dcode', 'dname']].copy()
    
    df_district_map['join_key'] = df_district_map['dname'].apply(normalize_district_name_v4_2)
    df_district_map = df_district_map.drop_duplicates(subset=['join_key'])
    
    print(f"✅ สร้างตารางจับคู่ DCODE สำเร็จ (มี {len(df_district_map)} เขตที่ไม่ซ้ำกัน)")

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดในการโหลด dcode map: {e}")
    raise e


# --- 3. ค้นหาและประมวลผลไฟล์ฝน 12 เดือน ---
print("\n--- 🌦️ ขั้นตอนที่ 2: เริ่มประมวลผลไฟล์ข้อมูลฝน ---")

rain_data_dir = '../data/RAW/rain_2024/'
all_xlsx_files = glob.glob(os.path.join(rain_data_dir, '*.xlsx'))

# กรองไฟล์ temporary ของ Excel (ที่ขึ้นต้นด้วย ~$) ออก
rain_files = sorted(
    [f for f in all_xlsx_files if not os.path.basename(f).startswith('~$')]
)

if not rain_files:
    print(f"❌ ไม่พบไฟล์ .xlsx ที่เป็นข้อมูลจริง ในโฟลอร์: {rain_data_dir}")
else:
    print(f"📂 พบไฟล์ข้อมูลฝน {len(rain_files)} ไฟล์ (กรองไฟล์ Temp แล้ว). กำลังเริ่มวนลูป...")

all_processed_data = []
start_time = time.time()

try:
    # Loop 1: วนลูปไฟล์
    for file_path in rain_files:
        file_name = os.path.basename(file_path)
        print(f"\n   กำลังประมวลผลไฟล์: {file_name}...")
        
        dfs_by_day = pd.read_excel(file_path, sheet_name=None)
        
        # Loop 2: วนลูปชีต
        for sheet_name, df_day in dfs_by_day.items():
            if df_day.empty or 'เขต' not in df_day.columns:
                print(f"      - (ข้าม) ชีต '{sheet_name}' ว่างเปล่า หรือไม่มีคอลัมน์ 'เขต'")
                continue
            
            # Transform & Merge
            df_day['join_key'] = df_day['เขต'].apply(normalize_district_name_v4_2)
            df_merged = pd.merge(
                df_day, df_district_map, on='join_key', how='left'
            )
            all_processed_data.append(df_merged)
            
        print(f"   ✅ {file_name} (รวม {len(dfs_by_day)} ชีต) ประมวลผลเสร็จสิ้น")


    # --- 4. รวมร่าง (Concat) ข้อมูลทั้งหมด ---
    if not all_processed_data:
        print("❌ ไม่พบข้อมูลใดๆ ให้รวมเลย!")
    else:
        print("\n" + "="*50)
        print("...กำลังรวมข้อมูลทั้งหมดจาก 365+ ชีต...")
        
        final_rain_df_raw = pd.concat(all_processed_data, ignore_index=True)
        
        end_time = time.time()
        print(f"✅ รวมข้อมูลทั้งหมดเสร็จสิ้น! (ใช้เวลา {end_time - start_time:.2f} วินาที)")
        print(f"   ข้อมูลดิบ (ก่อนกรอง): {final_rain_df_raw.shape[0]} แถว")

        
        # --- 5. [V4.3] กรองข้อมูลนอกพื้นที่ (สมุทรปราการ) ออก ---
        print("\n--- 🗑️ ขั้นตอนที่ 3: กรองข้อมูลนอกพื้นที่ กทม. ---")
        
        final_rain_df = final_rain_df_raw[
            final_rain_df_raw['เขต'] != 'อำเภอเมืองสมุทรปราการ'
        ].copy() 
        
        removed_rows = len(final_rain_df_raw) - len(final_rain_df)
        print(f"   ✅ กรอง 'อำเภอเมืองสมุทรปราการ' ออกไป {removed_rows} แถว")
        print(f"   ข้อมูลหลังกรอง (เฉพาะ กทม.): {final_rain_df.shape[0]} แถว")


        # --- 6. ตรวจสอบ และ [V.7] แปลง DCODE ---
        unmatched_rows = final_rain_df[final_rain_df['dcode'].isna()]
        unmatched_count = len(unmatched_rows)
        
        if unmatched_count > 0:
            print(f"\n   ⚠️ คำเตือน: ยังคงมี {unmatched_count} แถวที่หา DCODE ไม่พบ")
            unmatched_districts = unmatched_rows['เขต'].unique()
            print(f"   ตัวอย่างชื่อเขตที่หาไม่พบ: {unmatched_districts[:10]}")
        else:
            print("\n   ✅ DCODE ทั้งหมด (ใน กทม.) ถูกจับคู่เรียบร้อย!")
            
            # --- 🌟 [อัปเดต V.7] แปลง DCODE เป็นเลขจำนวนเต็ม ---
            # ทำขั้นตอนนี้หลังจากกรองแถว null (สมุทรปราการ) ออกแล้ว
            # และมั่นใจว่าไม่มี null เหลือแล้ว
            try:
                final_rain_df['dcode'] = final_rain_df['dcode'].astype(int)
                print("   - [V.7] แปลง dcode เป็นเลขจำนวนเต็ม (int) สำเร็จ")
            except ValueError as e:
                print(f"   - [!] ไม่สามารถแปลง dcode เป็น int ได้: {e}")
            
            
        # --- 7. จัดเรียงคอลัมน์และลบ 'ดูข้อมูล' ---
        
        final_rain_df_sorted = final_rain_df # สร้างตัวแปรใหม่เผื่อไว้
        
        if 'dcode' in final_rain_df.columns:
            # รายการคอลัมน์สำคัญที่เราต้องการเก็บไว้
            key_cols = ['dcode', 'dname', 'เขต', 'รหัสสถานี', 'สถานี', 'วัน-เวลา']
            
            # [อัปเดต] กรองคอลัมน์ที่ไม่ต้องการออก ('join_key' และ 'ดูข้อมูล')
            other_cols = [
                col for col in final_rain_df.columns 
                if col not in key_cols and col not in ['join_key', 'ดูข้อมูล']
            ]
            
            # รวมคอลัมน์สำคัญ + คอลัมน์ที่เหลือ (ที่กรองแล้ว)
            final_rain_df_sorted = final_rain_df[key_cols + other_cols]
            print("\n   [i] จัดเรียงคอลัมน์ใหม่ และลบคอลัมน์ 'ดูข้อมูล' แล้ว")

        # --- 8. บันทึกไฟล์ ---
        output_dir = '../data/PROCESSED/'
        output_filename = 'rain_2024_combined_bkk_only.csv' 
        output_path = os.path.join(output_dir, output_filename)
        
        os.makedirs(output_dir, exist_ok=True)
        
        print(f"\n💾 กำลังบันทึกไฟล์ผลลัพธ์ (เฉพาะ กทม.) ไปที่: {output_path}")
        final_rain_df_sorted.to_csv(output_path, index=False, encoding='utf-8-sig')
        print("🎉 บันทึกไฟล์สำเร็จ!")

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดร้ายแรงระหว่างประมวลผล: {e}")

--- 🗺️ ขั้นตอนที่ 1: กำลังสร้างตารางจับคู่ DCODE ---
✅ สร้างตารางจับคู่ DCODE สำเร็จ (มี 50 เขตที่ไม่ซ้ำกัน)

--- 🌦️ ขั้นตอนที่ 2: เริ่มประมวลผลไฟล์ข้อมูลฝน ---
📂 พบไฟล์ข้อมูลฝน 12 ไฟล์ (กรองไฟล์ Temp แล้ว). กำลังเริ่มวนลูป...

   กำลังประมวลผลไฟล์: 01.xlsx...
   ✅ 01.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 02.xlsx...
   ✅ 02.xlsx (รวม 29 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 03.xlsx...
   ✅ 03.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 04.xlsx...
   ✅ 04.xlsx (รวม 30 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 05.xlsx...
   ✅ 05.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 06.xlsx...
   ✅ 06.xlsx (รวม 30 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 07.xlsx...
   ✅ 07.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 08.xlsx...
   ✅ 08.xlsx (รวม 31 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 09.xlsx...
   ✅ 09.xlsx (รวม 30 ชีต) ประมวลผลเสร็จสิ้น

   กำลังประมวลผลไฟล์: 10.xlsx...
   ✅ 10.xlsx (รวม 31 ชีต) ประมวลผลเสร

In [6]:
import pandas as pd
import os
import re # สำหรับ Regular Expressions

# --- 1. [V4.2] ฟังก์ชันทำความสะอาดชื่อเขต (ตัวเก่งตัวเดิม) ---
def normalize_district_name_v4_2(name):
    """
    ฟังก์ชัน V4.2:
    1. แก้ไขคำย่อ/สะกดผิด (ป้อมปราบฯ, ราษฏร์บูรณะ)
    2. ลบคำนำหน้า (เขต, อำเภอ)
    3. ลบอักขระที่มองไม่เห็น (\xa0)
    4. ลบช่องว่างทุกชนิด ( \s+ ) ทิ้งทั้งหมด
    """
    if pd.isna(name):
        return None
    
    clean_name = str(name)
    clean_name = clean_name.replace('ป้อมปราบฯ', 'ป้อมปราบศัตรูพ่าย') # แก้คำย่อ
    clean_name = clean_name.replace('ราษฏร์บูรณะ', 'ราษฎร์บูรณะ')    # แก้ ฏ -> ฎ
    clean_name = clean_name.replace('เขต', '')
    clean_name = clean_name.replace('อำเภอ', '') 
    clean_name = clean_name.replace(u'\xa0', '')
    clean_name = re.sub(r'\s+', '', clean_name)
    
    return clean_name

# --- 2. 🌟 [แก้ไข] กำหนด Path ไปยังไฟล์ Excel หลัก ---

# (แนะนำ) ใช้ Relative Path (อ้างอิงจาก .ipynb)
file_path = '../data/PROCESSED/overall_data.xlsx'

# (ทางเลือก) ใช้ Absolute Path ที่คุณระบุมา
# file_path = '/Users/tk/Desktop/flowcast-flood-prediction/data/PROCESSED/overall_data.xlsx'

# 🌟 [แก้ไข] กำหนดชื่อ "ชีต" (Sheet) ที่เราต้องการ
district_sheet = 'districts'
risk_sheet = 'risk'
pump_sheet = 'pump_readiness'

# โฟลเดอร์ที่จะบันทึกไฟล์
output_dir = '../data/PROCESSED/'
os.makedirs(output_dir, exist_ok=True) # สร้างโฟลเดอร์ถ้ายังไม่มี

print("--- 🚀 เริ่มกระบวนการ V.5 (ฉบับแก้ไข - อ่านจากชีต Excel) ---")
print(f"ไฟล์เป้าหมาย: {file_path}")

try:
    # --- 3. สร้าง "แผนที่เขต" (District Map) ---
    print(f"\n🗺️ กำลังสร้าง 'แผนที่เขต' จากชีต: '{district_sheet}'")
    
    # 🌟 [แก้ไข] อ่านจากไฟล์ Excel และชีตที่ระบุ
    df_districts = pd.read_excel(file_path, sheet_name=district_sheet)
    
    df_district_map = df_districts[['dcode', 'dname']].copy()
    df_district_map['join_key'] = df_district_map['dname'].apply(normalize_district_name_v4_2)
    df_district_map = df_district_map.drop_duplicates(subset=['join_key'])
    
    print(f"   ✅ สร้าง 'แผนที่เขต' สำเร็จ (มี {len(df_district_map)} เขตแม่บท)")

    
    # --- 4. TASK 1: ประมวลผลชีต risk ---
    print("\n" + "="*50)
    print(f"🔧 TASK 1: กำลังเพิ่ม dcode ให้ชีต: '{risk_sheet}'")
    
    # 🌟 [แก้ไข] อ่านจากไฟล์ Excel และชีตที่ระบุ
    df_risk = pd.read_excel(file_path, sheet_name=risk_sheet)
    print(f"   - ข้อมูลดิบ {risk_sheet}: {df_risk.shape[0]} แถว")
    
    df_risk['join_key'] = df_risk['เขต'].apply(normalize_district_name_v4_2)
    
    df_risk_processed = pd.merge(
        df_risk.drop(columns=['join_key']), # ลบ key ทิ้งหลัง merge
        df_district_map.drop(columns=['join_key']), # ลบ key ทิ้งหลัง merge
        left_on=df_risk['join_key'], # ใช้ key ที่สร้าง
        right_on=df_district_map['join_key'], # ใช้ key ที่สร้าง
        how='left'
    ).drop(columns=['key_0']) # ลบคอลัมน์ key_0 ที่เกิดจากการ merge

    # 4. บันทึกไฟล์
    output_risk_path = os.path.join(output_dir, 'risk_with_dcode.csv')
    df_risk_processed.to_csv(output_risk_path, index=False, encoding='utf-8-sig')
    print(f"   💾 บันทึกไฟล์ risk ที่แก้ไขแล้วไปที่: {output_risk_path}")


    # --- 5. TASK 2: ประมวลผลชีต pump_readiness ---
    print("\n" + "="*50)
    print(f"🔧 TASK 2: กำลังแก้ไข dcode ในชีต: '{pump_sheet}'")
    
    # 🌟 [แก้ไข] อ่านจากไฟล์ Excel และชีตที่ระบุ
    df_pump = pd.read_excel(file_path, sheet_name=pump_sheet)
    print(f"   - ข้อมูลดิบ {pump_sheet}: {df_pump.shape[0]} แถว")
    
    # 1. [สำคัญ] ลบ dcode เก่าที่ "ผิด" ออก
    if 'dcode' in df_pump.columns:
        df_pump_corrected = df_pump.drop(columns=['dcode'])
        print("   - ลบ dcode เก่า (ที่ผิด) ออกแล้ว")
    else:
        df_pump_corrected = df_pump.copy()
        
    # 2. สร้าง join_key (จากคอลัมน์ "dname")
    df_pump_corrected['join_key'] = df_pump_corrected['dname'].apply(normalize_district_name_v4_2)
    
    # 3. Merge เพื่อดึง dcode และ dname ที่ "ถูกต้อง" จาก "แม่บท"
    df_pump_processed = pd.merge(
        df_pump_corrected,
        df_district_map, # .drop(columns=['dname']) # เอา dname จากแม่บทมาเลย
        on='join_key',
        how='left',
        suffixes=('_old', None) # ถ้ามี dname ซ้ำ ให้เก็บอันใหม่ (จากแม่บท)
    )

    # 4. จัดเรียงคอลัมน์ (เอา dcode, dname ใหม่มาไว้หน้าๆ)
    # กรองคอลัมน์จาก df_pump ออก (ไม่เอา 'dname_old', 'join_key')
    original_cols = [col for col in df_pump.columns if col not in ['dcode', 'dname']]
    
    final_cols = ['dcode', 'dname'] + original_cols
    final_cols_exist = [col for col in final_cols if col in df_pump_processed.columns]
    df_pump_processed = df_pump_processed[final_cols_exist]

    # 5. บันทึกไฟล์
    output_pump_path = os.path.join(output_dir, 'pump_readiness_corrected_dcode.csv')
    df_pump_processed.to_csv(output_pump_path, index=False, encoding='utf-8-sig')
    print(f"   💾 บันทึกไฟล์ pump ที่แก้ไขแล้วไปที่: {output_pump_path}")
    

    print("\n" + "="*50)
    print("🎉 กระบวนการ V.5 (ฉบับแก้ไข) เสร็จสิ้นทั้งหมด!")

except FileNotFoundError:
    print(f"❌ เกิดข้อผิดพลาด: ไม่พบไฟล์")
    print(f"กรุณาตรวจสอบว่าไฟล์นี้มีอยู่จริง: {file_path}")
except ValueError as e:
    # Error นี้มักจะเกิดถ้าชื่อชีต (sheet_name) ผิด
    print(f"❌ เกิดข้อผิดพลาด: {e}")
    print(f"   กรุณาตรวจสอบว่าชื่อชีต '{district_sheet}', '{risk_sheet}', '{pump_sheet}' ถูกต้อง")
except KeyError as e:
    print(f"❌ เกิดข้อผิดพลาด: ไม่พบคอลัมน์")
    print(f"ไม่พบคอลัมน์ชื่อ {e} (อาจจะเป็น 'เขต' หรือ 'dname')")
    print("กรุณาตรวจสอบชื่อคอลัมน์ในไฟล์ Excel ของคุณอีกครั้ง")
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดไม่คาดคิด: {e}")

--- 🚀 เริ่มกระบวนการ V.5 (ฉบับแก้ไข - อ่านจากชีต Excel) ---
ไฟล์เป้าหมาย: ../data/PROCESSED/overall_data.xlsx

🗺️ กำลังสร้าง 'แผนที่เขต' จากชีต: 'districts'
   ✅ สร้าง 'แผนที่เขต' สำเร็จ (มี 50 เขตแม่บท)

🔧 TASK 1: กำลังเพิ่ม dcode ให้ชีต: 'risk'
❌ เกิดข้อผิดพลาด: Worksheet named 'risk' not found
   กรุณาตรวจสอบว่าชื่อชีต 'districts', 'risk', 'pump_readiness' ถูกต้อง


In [7]:
import pandas as pd
import os
import re # สำหรับ Regular Expressions

# --- 1. [V.6] ฟังก์ชันทำความสะอาดชื่อเขต (ตัวเก่งตัวเดิม) ---
def normalize_V6(name):
    """
    ฟังก์ชัน V.6: แก้ไข Typos ('คลองสมวา', 'สำนักงนเขต', 'ป้อมปราบฯ', 'ราษฏร์บูรณะ')
    และลบคำนำหน้า/ช่องว่างทั้งหมด
    """
    if pd.isna(name):
        return None
    
    clean_name = str(name)
    
    # 1. จัดการ Typos/คำย่อ
    clean_name = clean_name.replace('ป้อมปราบฯ', 'ป้อมปราบศัตรูพ่าย')
    clean_name = clean_name.replace('ราษฏร์บูรณะ', 'ราษฎร์บูรณะ')
    clean_name = clean_name.replace('คลองสมวา', 'คลองสามวา')
    
    # 2. ลบคำนำหน้า (Prefixes)
    clean_name = clean_name.replace('สำนักงนเขต', '') 
    clean_name = clean_name.replace('สำนักงานเขต', '')
    clean_name = clean_name.replace('เขต', '')
    clean_name = clean_name.replace('อำเภอ', '') 
    
    # 3. จัดการแถวพิเศษ
    if 'สำนักการระบายน้ำ' in clean_name:
        return 'สำนักการระบายน้ำ' 
    
    # 4. ลบอักขระและช่องว่าง
    clean_name = clean_name.replace(u'\xa0', '')
    clean_name = re.sub(r'\s+', '', clean_name)
    
    return clean_name

# --- 2. กำหนด Path (เหมือนเดิม) ---
file_path = '../data/PROCESSED/overall_data.xlsx'
district_sheet = 'districts'
risk_sheet = 'risk'
pump_sheet = 'pump_readiness'

output_dir = '../data/PROCESSED/'
os.makedirs(output_dir, exist_ok=True)

print("--- 🚀 เริ่มกระบวนการ V.7 (แก้ไข dcode เป็นเลขจำนวนเต็ม) ---")
print(f"ไฟล์เป้าหมาย: {file_path}")

try:
    # --- 3. สร้าง "แผนที่เขต" (District Map) ---
    print(f"\n🗺️ กำลังสร้าง 'แผนที่เขต' จากชีต: '{district_sheet}'")
    
    df_districts = pd.read_excel(file_path, sheet_name=district_sheet)
    
    df_district_map = df_districts[['dcode', 'dname']].copy()
    df_district_map['join_key'] = df_district_map['dname'].apply(normalize_V6)
    df_district_map = df_district_map.drop_duplicates(subset=['join_key'])
    
    print(f"   ✅ สร้าง 'แผนที่เขต' (V.6) สำเร็จ")

    
    # --- 4. TASK 1: ประมวลผลชีต risk ---
    print("\n" + "="*50)
    print(f"🔧 TASK 1: กำลังเพิ่ม dcode ให้ชีต: '{risk_sheet}'")
    
    df_risk = pd.read_excel(file_path, sheet_name=risk_sheet)
    df_risk['join_key'] = df_risk['เขต'].apply(normalize_V6)
    
    df_risk_processed = pd.merge(
        df_risk,
        df_district_map,
        on='join_key',
        how='left',
        suffixes=('_old', None)
    )
    
    # 🌟 [V.7] แปลง dcode เป็น int (สันนิษฐานว่า risk ไม่มีแถว null)
    if 'dcode' in df_risk_processed.columns and df_risk_processed['dcode'].isna().sum() == 0:
        df_risk_processed['dcode'] = df_risk_processed['dcode'].astype(int)
        print("   - [V.7] แปลง dcode ของ risk เป็นเลขจำนวนเต็ม (int) สำเร็จ")
    
    # บันทึกไฟล์ risk
    output_risk_path = os.path.join(output_dir, 'risk_with_dcode.csv')
    df_risk_processed.to_csv(output_risk_path, index=False, encoding='utf-8-sig')
    print(f"   💾 บันทึกไฟล์ risk ที่แก้ไขแล้ว (V.7) ไปที่: {output_risk_path}")


    # --- 5. TASK 2: ประมวลผลชีต pump_readiness ---
    print("\n" + "="*50)
    print(f"🔧 TASK 2: กำลังแก้ไข dcode ในชีต: '{pump_sheet}'")
    
    df_pump = pd.read_excel(file_path, sheet_name=pump_sheet)
    
    df_pump_corrected = df_pump.drop(columns=['dcode'], errors='ignore')
    print(f"   - ข้อมูลดิบ {pump_sheet}: {df_pump.shape[0]} แถว (ลบ dcode เก่าแล้ว)")
        
    df_pump_corrected['join_key'] = df_pump_corrected['dname'].apply(normalize_V6)
    
    df_pump_processed = pd.merge(
        df_pump_corrected,
        df_district_map,
        on='join_key',
        how='left',
        suffixes=('_old', None)
    )
    
    # 1. ลบแถว null (สำนักการระบายน้ำ)
    df_pump_final = df_pump_processed.dropna(subset=['dcode']).copy()
    removed_count = len(df_pump_processed) - len(df_pump_final)
    print(f"   - ลบแถวที่ dcode เป็น null ออก {removed_count} แถว")

    # 2. 🌟 [V.7] แปลง dcode เป็น int (หลังจากลบ null แล้ว)
    df_pump_final['dcode'] = df_pump_final['dcode'].astype(int)
    print("   - [V.7] แปลง dcode ของ pump เป็นเลขจำนวนเต็ม (int) สำเร็จ")

    # 3. จัดเรียงคอลัมน์
    original_cols = [col for col in df_pump.columns if col not in ['dcode', 'dname']]
    final_cols = ['dcode', 'dname'] + original_cols
    final_cols_exist = [col for col in final_cols if col in df_pump_final.columns]
    df_pump_processed_sorted = df_pump_final[final_cols_exist] 

    # 4. บันทึกไฟล์
    output_pump_path = os.path.join(output_dir, 'pump_readiness_corrected_dcode.csv')
    df_pump_processed_sorted.to_csv(output_pump_path, index=False, encoding='utf-8-sig')
    print(f"   💾 บันทึกไฟล์ pump ที่แก้ไขแล้ว (V.7) ไปที่: {output_pump_path}")
    

    print("\n" + "="*50)
    print("🎉 กระบวนการ V.7 เสร็จสิ้นทั้งหมด!")

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")
    print("กรุณาตรวจสอบ Path, ชื่อชีต, และชื่อคอลัมน์อีกครั้ง")

--- 🚀 เริ่มกระบวนการ V.7 (แก้ไข dcode เป็นเลขจำนวนเต็ม) ---
ไฟล์เป้าหมาย: ../data/PROCESSED/overall_data.xlsx

🗺️ กำลังสร้าง 'แผนที่เขต' จากชีต: 'districts'
   ✅ สร้าง 'แผนที่เขต' (V.6) สำเร็จ

🔧 TASK 1: กำลังเพิ่ม dcode ให้ชีต: 'risk'
❌ เกิดข้อผิดพลาด: Worksheet named 'risk' not found
กรุณาตรวจสอบ Path, ชื่อชีต, และชื่อคอลัมน์อีกครั้ง


In [11]:
import pandas as pd
import os

# --- 1. นิยามฟังก์ชันสำหรับแบ่งฤดู ---
# (ฟังก์ชันนี้รับ Date object (ที่เป็น ค.ศ. แล้ว) เป็น input)
def get_season(date):
    month = date.month
    day = date.day
    
    # ฤดูร้อน (ประมาณ 16 ก.พ. - 15 พ.ค.)
    if (month == 2 and day >= 16) or (month in [3, 4]) or (month == 5 and day <= 15):
        return 'Summer'
    # ฤดูฝน (ประมาณ 16 พ.ค. - 15 ต.ค.)
    elif (month == 5 and day >= 16) or (month in [6, 7, 8, 9]) or (month == 10 and day <= 15):
        return 'Rainy'
    # ฤดูหนาว (ที่เหลือ)
    else:
        return 'Winter'

# --- 2. โหลดไฟล์ฝนเดิม ---
# (สำคัญ!) ตรวจสอบว่า Path นี้ถูกต้อง จากตำแหน่ง Notebook ของคุณ
rain_file_path = '../data/PROCESSED/rain_2024_combined_bkk_only.csv' 
print(f"--- 1. กำลังโหลดไฟล์: {rain_file_path} ---")

try:
    rain_df = pd.read_csv(rain_file_path)
    print(f"   ... โหลดสำเร็จ {len(rain_df)} แถว")
except FileNotFoundError:
    print(f"!!! Error: ไม่พบไฟล์ {rain_file_path}")
    raise
except Exception as e:
    print(f"เกิดข้อผิดพลาด: {e}")
    raise

# --- 3. [แก้ไขใหม่] ทำ Feature Engineering (ซ่อมวันที่ พ.ศ.) ---
print("--- 2. กำลังแปลงวันที่ พ.ศ. 2567 เป็น ค.ศ. 2024 ... ---")
try:
    # 1. แปลงคอลัมน์ 'วัน-เวลา' เป็น string (เพื่อให้แน่ใจว่า .str ได้)
    date_str = rain_df['วัน-เวลา'].astype(str)
    
    # 2. (สำคัญ) แทนที่ '2567' (พ.ศ.) ด้วย '2024' (ค.ศ.)
    corrected_date_str = date_str.str.replace('2567', '2024')
    
    # 3. (สำคัญ) แปลง String ค.ศ. ใหม่ ให้เป็น datetime object
    #    - dayfirst=True: เพื่อให้มันอ่าน '29/01/2024' (DD/MM/YYYY) ได้ถูกต้อง
    #    - errors='coerce': ถ้าเจอวันที่พังจริงๆ (เช่น 'nan', '29/0') ให้แปลงเป็น NaT (Not a Time)
    rain_df['date_object'] = pd.to_datetime(corrected_date_str, dayfirst=True, errors='coerce')
    
    # 4. (ใหม่) ตรวจสอบและลบแถวที่วันที่พัง (NaT)
    original_rows = len(rain_df)
    rain_df.dropna(subset=['date_object'], inplace=True)
    dropped_rows = original_rows - len(rain_df)
    
    if dropped_rows > 0:
        print(f"   ... (คำเตือน) ลบแถวที่แปลงวันที่ไม่ได้ {dropped_rows} แถว")
    print("   ... แปลงเป็น datetime object (ค.ศ.) สำเร็จ!")

except Exception as e:
    print(f"!!! Error ระหว่างการแปลงวันที่: {e}")
    raise

# --- 4. สร้างคอลัมน์ 'season' ---
rain_df['season'] = rain_df['date_object'].apply(get_season)
print("--- 3. สร้างคอลัมน์ 'season' สำเร็จ ---")
print(rain_df[['วัน-เวลา', 'date_object', 'season']].head())

# --- 5. บันทึกไฟล์ใหม่ ---
output_path = '../data/PROCESSED/rain_2024_with_seasons.csv'
rain_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"--- 4. บันทึกไฟล์ใหม่ไปที่ {output_path} เรียบร้อย! ---")

--- 1. กำลังโหลดไฟล์: ../data/PROCESSED/rain_2024_combined_bkk_only.csv ---
   ... โหลดสำเร็จ 18300 แถว
--- 2. กำลังแปลงวันที่ พ.ศ. 2567 เป็น ค.ศ. 2024 ... ---
   ... (คำเตือน) ลบแถวที่แปลงวันที่ไม่ได้ 11100 แถว
   ... แปลงเป็น datetime object (ค.ศ.) สำเร็จ!
--- 3. สร้างคอลัมน์ 'season' สำเร็จ ---
              วัน-เวลา date_object  season
0  2567-01-01 00:00:00  2024-01-01  Winter
1  2567-01-01 00:00:00  2024-01-01  Winter
2  2567-01-01 00:00:00  2024-01-01  Winter
3  2567-01-01 00:00:00  2024-01-01  Winter
4  2567-01-01 00:00:00  2024-01-01  Winter
--- 4. บันทึกไฟล์ใหม่ไปที่ ../data/PROCESSED/rain_2024_with_seasons.csv เรียบร้อย! ---


In [ ]:
import pandas as pd
import os

print("--- Correlation Matrix Export Script Started ---")

# --- 1. Define Paths ---
# Path to the source data (assuming notebook is in 'notebooks' folder)
input_csv_path = '../data/PROCESSED/master_district_features.csv'

# Path to the frontend public data directory
output_json_dir = '../frontend/public/data'
output_json_path = os.path.join(output_json_dir, 'correlation_matrix.json')

# Ensure the output directory exists
os.makedirs(output_json_dir, exist_ok=True)

print(f"Loading data from: {input_csv_path}")

try:
    # --- 2. Load Data ---
    df = pd.read_csv(input_csv_path)

    # --- 3. Select Features ---
    features_to_correlate = [
        'pump_capacity_total', 
        'canal_count', 
        'num_risk_points', 
        'AREA'
    ]
    
    # Check if all columns exist
    missing_cols = [col for col in features_to_correlate if col not in df.columns]
    if missing_cols:
        print(f"Error: Missing columns in the CSV: {missing_cols}")
    else:
        df_features = df[features_to_correlate]
        
        # --- 4. Calculate Correlation ---
        # Using Pearson correlation
        corr_matrix = df_features.corr(method='pearson')
        
        print("\n--- Calculated Correlation Matrix ---")
        print(corr_matrix)
        
        # --- 5. Export to JSON ---
        # Use orient='index' for a clean key-value structure: {row: {col: value}}
        # This format is very easy for JavaScript/React to read
        corr_matrix.to_json(output_json_path, orient='index', indent=4)
        
        print(f"\n[SUCCESS] Successfully exported correlation matrix to:")
        print(output_json_path)

except FileNotFoundError:
    print(f"Error: Input file not found at {input_csv_path}")
    print("Please check the path and run the script again.")
except Exception as e:
    print(f"An error occurred: {e}")

print("--- Script Finished ---")

SyntaxError: invalid syntax (2851573667.py, line 71)

In [2]:
import pandas as pd
pd.set_option('display.max_rows', None) # แสดงทุกคอลัมน์
df_check = pd.read_csv('../data/PROCESSED/master_district_features.csv')
print("--- Actual Columns in master_district_features.csv ---")
print(list(df_check.columns))

--- Actual Columns in master_district_features.csv ---
['OBJECTID', 'AREA', 'dcode', 'dname', 'dname_e', 'pcode', 'pname', 'num_male', 'num_female', 'num_school', 'num_hos', 'num_comm', 'num_temple', 'num_health', 'geometry', 'population', 'canal_count', 'flood_point_count', 'floodgate_count', 'avg_rain_rainy', 'avg_rain_summer', 'avg_rain_winter', 'district_group', 'budget_year', 'inspec_period', 'pump_number', 'pump_ready', 'pump_repair', 'pump_sell', 'pump_support', 'total_ready', 'คะแนนรวม', 'จำนวนจุดเสี่ยง', 'จำนวนจุดเฝ้าระวังน้ำท่วม']


In [4]:
import pandas as pd
import os

print("--- Correlation Matrix Export Script (V2 - Updated) ---")

# --- [!!!] กรุณาแก้ไขลิสต์นี้ [!!!] ---
#
# ชื่อคอลัมน์ 'pump_capacity_total' และ 'num_risk_points' จากแผนเดิมของเรา "ไม่มีอยู่จริง" ในไฟล์ CSV
#
# 1. รบกวนดูผลลัพธ์จากโค้ด (print(list(df_check.columns))) ในขั้นตอนที่ 1
# 2. นำชื่อคอลัมน์ที่ถูกต้องมาแทนที่ในลิสต์ 'features_to_correlate' นี้
#
# ตัวอย่าง: ถ้าชื่อที่ถูกต้องคือ 'pump_total' และ 'num_risk'
# features_to_correlate = [
#     'pump_total', 
#     'canal_count', 
#     'num_risk', 
#     'AREA'
# ]
# -----------------------------------------------------------------
features_to_correlate = [
    'YOUR_CORRECT_PUMP_COLUMN_NAME',  # <-- FIXME: ใส่ชื่อคอลัมน์ "กำลังสูบ" ที่ถูกต้อง
    'canal_count',                     # (อันนี้น่าจะถูกแล้ว)
    'YOUR_CORRECT_RISK_COLUMN_NAME', # <-- FIXME: ใส่ชื่อคอลัมน์ "จุดเสี่ยง" ที่ถูกต้อง
    'AREA'                           # (อันนี้น่าจะถูกแล้ว)
]
# -----------------------------------------------------------------


# --- 1. Define Paths ---
input_csv_path = '../data/PROCESSED/master_district_features.csv'
output_json_dir = '../frontend/public/data'
output_json_path = os.path.join(output_json_dir, 'correlation_matrix.json')

os.makedirs(output_json_dir, exist_ok=True)
print(f"Loading data from: {input_csv_path}")

try:
    # --- 2. Load Data ---
    df = pd.read_csv(input_csv_path)

    # --- 3. Check Features ---
    missing_cols = [col for col in features_to_correlate if col not in df.columns]
    
    if missing_cols:
        print(f"\n[ERROR] Still missing columns: {missing_cols}")
        print("Please check your spelling in the 'features_to_correlate' list.")
        print("\nAvailable columns are:")
        print(list(df.columns))
    else:
        df_features = df[features_to_correlate]
        
        # --- 4. Calculate Correlation ---
        corr_matrix = df_features.corr(method='pearson')
        
        print("\n--- Calculated Correlation Matrix ---")
        print(corr_matrix)
        
        # --- 5. Export to JSON ---
        corr_matrix.to_json(output_json_path, orient='index', indent=4)
        
        print(f"\n[SUCCESS] Successfully exported correlation matrix to:")
        print(output_json_path)

except FileNotFoundError:
    print(f"Error: Input file not found at {input_csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

print("--- Script Finished ---")


--- Correlation Matrix Export Script (V2 - Updated) ---
Loading data from: ../data/PROCESSED/master_district_features.csv

[ERROR] Still missing columns: ['YOUR_CORRECT_PUMP_COLUMN_NAME', 'YOUR_CORRECT_RISK_COLUMN_NAME']
Please check your spelling in the 'features_to_correlate' list.

Available columns are:
['OBJECTID', 'AREA', 'dcode', 'dname', 'dname_e', 'pcode', 'pname', 'num_male', 'num_female', 'num_school', 'num_hos', 'num_comm', 'num_temple', 'num_health', 'geometry', 'population', 'canal_count', 'flood_point_count', 'floodgate_count', 'avg_rain_rainy', 'avg_rain_summer', 'avg_rain_winter', 'district_group', 'budget_year', 'inspec_period', 'pump_number', 'pump_ready', 'pump_repair', 'pump_sell', 'pump_support', 'total_ready', 'คะแนนรวม', 'จำนวนจุดเสี่ยง', 'จำนวนจุดเฝ้าระวังน้ำท่วม']
--- Script Finished ---
